# Machine Learning prediction, Econometric modelisation and Economic analysis applied to Cardiovascular diseases



  * This file is part of project Creation-of-a-search-engine. 
    It's copyrighted by the contributors
  * Copyright (c) 2021 lprtk

##### Cardiovascular Disease dataset : https://www.kaggle.com/sulianova/cardiovascular-disease-dataset

Can statistical modelling, through estimation and prediction, serve as a decision criterion to ensure effective medical prevention? Giving a cost to health, to choose the most cost-effective treatment is very complicated. Our medical economic analysis first seeks to identify the most important risk factors in the development of cardiovascular diseases. Then, in a second phase, to know how and to whom should be the prevention of these diseases.

##### Variables & Informations

    - Id : It is an identifier assigned to each patient.
 
    - Age : This is the age of each patient in days.

    - Gender : Binary variable indicating the gender of each individual (Male = 2 or Female = 1).

    - Height : This is the height of the patient (in Cm).

    - Weight : This is the weight of the patient (in Kg).

    - Ap_hi : Systolic blood pressure. During the systolic phase (in Mmhg).

    - Ap_lo : Diastolic Blood Pressure. During the diastole phase (in Mmhg).

    - Cholesterol : This is a type of fat found in the blood normally expressed in mg/dl. But here, we have only levels: 1 corresponding to the lowest (so-called normal) amount and 3 to the highest (so-called very high).\\

    - Glucose : It is the level of Glucose in the blood, expressed also in level 1 to 3.

    - Smoke : It is a binary variable that indicates whether the individual smokes or not (Non-smoking = 0 or Smoking = 1).

    - Alcool : It is a binary variable that indicates whether the patient is an alcoholic or not (Non-alcoholic = 0 or Alcoholic = 1).

    - Active : It is a binary variable that indicates whether the patient is physically active or not (No = 0 or Yes = 1).

    - Cardio : It is a binary variable that indicates whether an patient has cardiovascular disease (CVD) or not (Has no CVD = 0 Has a CVD = 1).

##### Additional variables created & Informations

    - BMI : This is the body mass index. It is classified according to 4 levels: less than 18.5, between 18.5 and 24, between 25 and 29, then greater than or equal to 30.

    - BPC : This is blood pressure control. It is categorized into 5 levels: to inf 120mmgh to sup at 180mmgh for Systolic. To inf 80mmgh to sup 12mmgh for Diastolic.


#### Librairies and data loading

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

import pandas.util.testing as tm

from scipy import stats
from scipy.stats import chi2_contingency

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression, SGDClassifier 
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score 

from sklearn.metrics import classification_report, plot_roc_curve, confusion_matrix 

path = input("Entrer le chemin d'accès aux fichier : ")
file = input("Entrer le nom du fichier : ")
data = pd.read_csv(path+enter_file, sep = ";")

sns.set_style('darkgrid')
background_color = ['#eaeaf2']
my_palette = ['#FA5858',
              '#FA8258',
              '#F7D358',
              '#ACFA58',
              '#58D3F7',
              '#5858FA',
              '#BE81F7']
sns.set_palette(my_palette, 7)

#### First impressions of the dataset

In [ ]:
data.shape

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.describe()

## 1. Data Cleaning

##### Some columns are renamed

In [ ]:
data.rename(columns={'ap_hi': 'systolic', 'ap_lo': 'diastolic', 'gluc': 'glucose', 
                   'alco': 'alcohol', 'cardio': 'cardiovascular disease'}, inplace=True)

##### 1) We remove the column 'Id' which is useless.
##### 2) We look if there are missing values in the dataset.
##### 3) We look if there are duplicated values (possible because we have removed the 'Id' column).
##### 4) We transform the column 'Age' into years and display additional information.
##### 5) We transform the variable 'Gender' into binary, we re-code it into 0 and 1.

In [ ]:
data.drop(['id'], axis=1, inplace= True)

In [ ]:
print("Il y a {} valeurs manquantes dans le dataset".format(data.isnull().sum().sum()))
print("Il y a {} valeurs dupliquées dans le dataset".format(data.duplicated().sum()))

In [ ]:
data.drop_duplicates(inplace=True)
print("Il y a {} valeurs dupliquées dans le dataset".format(data.duplicated().sum()))

In [ ]:
data.loc[:, 'age'] = data.loc[:, 'age'].apply(lambda x: int(x/365))
print('Min Age: ', data['age'].min())
print('Max Age: ', data['age'].max())
print('Mean Age: ', data['age'].mean())

In [ ]:
data.gender = data.gender.replace(2,0)

##### 6) We will create a new variable BMI

In [ ]:
data["bmi"] = data["weight"] / (data["height"]/100)**2

##### 7) We are going to classify the BMI of individuals by level

In [ ]:
data['bmi_class'] = 0
data.loc[(data['bmi'] < 18.5), 'bmi_class'] = 1
data.loc[(data['bmi'] >= 18.5) & (data['bmi'] < 25), 'bmi_class'] = 2
data.loc[(data['bmi'] >= 25) & (data['bmi'] < 30), 'bmi_class'] = 3
data.loc[(data['bmi'] >= 30), 'bmi_class'] = 4

##### 8) Now we need to remove the outliers of blood pressure, weight and height:
The normal systolic blood pressure is between 90 and 120.'

Similarly, diastolic blood pressure should be between 60 and 90 for a healthy individual. '

We are going to remove extremely rare cases of size and weight. Since the data is quite large, there will be no problem during modeling.

In [ ]:
# Remove Systolic and Diastolic blood pressiure outliers
out_filter = ((data["ap_hi"]>250) | (data["ap_lo"]>200))
print("There is {} outlier".format(data[out_filter]["cardio"].count()))
data = data[~out_filter]

out_filter2 = ((data["ap_hi"]<50) | (data["ap_lo"]<50))
print("There is {} outlier".format(data[out_filter2]["cardio"].count()))
data = data[~out_filter2]

data = data[data['ap_hi'] > data['ap_lo']].reset_index(drop=True)

##### Tails can be removed from CI distributions of patient weight and height

In [ ]:
weight_min_outlier_mask = data['weight'] > data['weight'].quantile(0.005)
weight_max_outlier_mask = data['weight'] < data['weight'].quantile(0.975)
data = data[(weight_min_outlier_mask) & (weight_max_outlier_mask)]

height_min_outlier_mask = data['height'] > data['height'].quantile(0.005)
height_max_outlier_mask = data['height'] < data['height'].quantile(0.975)
data = data[(height_min_outlier_mask) & (height_max_outlier_mask)]

##### Body mass index outliers are also removed.

In [ ]:
out_filter4 = ((data["bmi"]>72) | (data["bmi"]<=14))
print("There is {} outlier".format(data[out_filter4]["cardiovascular disease"].count()))
data = data[~out_filter4]

##### Once the outliers are removed, we will classify the blood pressure levels with a variable 'blood pressure control'.

In [ ]:
data['bpc'] = 0
data.loc[(data['systolic'] < 120) & (data['diastolic'] < 80), 'bpc'] = 1
data.loc[((data['systolic'] >= 120) & (data['systolic'] < 130)) &
         ((data['diastolic'] < 80)), 'bpc'] = 2
data.loc[((data['systolic'] >= 130) & (data['systolic'] < 140)) |
         ((data['diastolic'] >= 80) & (data['diastolic'] < 90)), 'bpc'] = 3
data.loc[((data['systolic'] >= 140) & (data['systolic'] < 180)) |
         ((data['diastolic'] >= 90) & (data['diastolic'] < 120)), 'bpc'] = 4
data.loc[(data['systolic'] >= 180) | (data['diastolic'] >= 120), 'bpc'] = 5

##### Let's take a look at the status of our data now:

In [ ]:
print('Total {} datapoints remaining with {} features'.format(data.shape[0], data.shape[1]))
print("Tous les outliers ont été suprrimés")
data

## 2. Data Visualization

#### 2.1. Distribution of men and women in the data

In [ ]:
sns.countplot(x= 'gender', hue= 'cardiovascular disease', data= data,)
plt.title('Comparation - {}'.format('gender'))
plt.legend(['Femmes (0)', 'Hommes (1)'], loc= 'upper right')

#### 2.2. What about the variable 'age'?

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))
sns.distplot(data['age'])
plt.xlim(data['age'].min(), data['age'].max())
plt.title('Comparation - {}'.format('age'))
plt.subplots_adjust(hspace= 0.6)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))
sns.countplot(x= 'age', hue= 'cardiovascular disease', data= data)
plt.title('CD - Cardiovascular Disease')
plt.legend(['CD (0)', 'CD (1)'], loc= 'upper right')

plt.subplots_adjust(hspace= 0.3)
plt.show()

#### 2.3. Distribution of weights and heights of individuals

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))

plt.subplot(2,2,1)
sns.distplot(data['height'])
plt.title('Distribtion')

plt.subplot(2,2,2)
sns.distplot(data['height'], kde= False, )
plt.xlim(140, 190)
plt.title('Zoom / No KDE')

plt.subplot(2,2,3)
sns.distplot(data['weight'])
plt.title('Distribtion')

plt.subplot(2,2,4)
sns.distplot(data['weight'], kde= False)
plt.xlim(40, 130)
plt.title('Zoom / No KDE')

plt.subplots_adjust(hspace= 0.6)
plt.show()

In [ ]:
fig, (ax1) = plt.subplots(1,1, figsize=(10,10))
sns.boxenplot(data['cardiovascular disease'],(data['height']),ax=ax1)
ax1.set_title('Height / Diseased')
plt.show()

#### 2.4. Distribution of individuals with different Glucose and Cholesterol levels

In [ ]:
plt.subplot(2,1,1)
sns.countplot('cholesterol', data= data)
plt.title('Cholesterol')

plt.subplot(2,1,2)
sns.countplot('glucose', data= data)
plt.title('Glucose')

plt.subplots_adjust(hspace= 0.6)
plt.show()

#### 2.5. Visualization of smokers, alcoholics, sprotics and overall individuals with cardiovascular disease.

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))

plt.subplot(2,2,1)
sns.countplot('smoke', data= data)
plt.title('Smoke')

plt.subplot(2,2,2)
sns.countplot('alcohol', data= data)
plt.title('Alcohol')

plt.subplot(2,2,3)
sns.countplot('active', data= data)
plt.title('Activity')

plt.subplot(2,2,4)
sns.countplot('cardiovascular disease', data= data)
plt.title('Cardio disease')

plt.subplots_adjust(hspace= 0.6)
plt.show()

In [ ]:
data['smoke/drink'] = data['smoke'].apply(str)+'|'+data['alcohol'].apply(str)
tmp = pd.crosstab(data['smoke/drink'],data['cardiovascular disease'],normalize='index')
tmp.reset_index()
tmp.columns = ['Not diseased','diseased']
sns.countplot(data['smoke/drink'],order=list(tmp.index))
plt.title("Nombre de patients atteint d'une maladie cardiovasculaire")
plt.xlabel('Fumeur/Alcohol : \n 0|0: non fumeur et non alcoholique ; 0|1: non fumeur alcoholique ; 1|0: fumeur et non alcoholique ; 1|1: fumeur et alcoholique')

#### 2.6. Complete data visualization of the dataset

In [ ]:
def habitPlot(dataframe, col):
    sns.countplot(x= col,
                  hue= 'cardiovascular disease',
                  data= data,
                  palette= my_palette)
    plt.title('Comparation - {}'.format(col))
    plt.legend(['CD (0)', 'CD (1)'],
               loc= 'upper right')

In [ ]:
fig, ax = plt.subplots(figsize = (10, 12))
fig.suptitle('CD -> Cardiovasculare Disease')

plt.subplot(4,2,1)
habitPlot(data, 'gender')

plt.subplot(4,2,2)
habitPlot(data, 'cholesterol')

plt.subplot(4,2,3)
habitPlot(data, 'glucose')

plt.subplot(4,2,4)
habitPlot(data, 'smoke')

plt.subplot(4,2,5)
habitPlot(data, 'alcohol')

plt.subplot(4,2,6)
habitPlot(data, 'active')

plt.subplot(4,2,7)
habitPlot(data, 'bmi_class')

plt.subplot(4,2,8)
habitPlot(data, 'bpc')

plt.subplots_adjust(hspace= 0.6, wspace= 0.3)
plt.show()

#### 2.7. Complete dataset visualization

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))

plt.subplot(3,2,1)
sns.boxenplot(data['cardiovascular disease'],(data['weight']))
plt.title('Weight')

plt.subplot(3,2,2)
sns.boxenplot(data['cardiovascular disease'],(data['height']))
plt.title('Height')

plt.subplot(3,2,3)
sns.boxenplot(data['cardiovascular disease'],(data['systolic']))
plt.title('Systolic')

plt.subplot(3,2,4)
sns.boxenplot(data['cardiovascular disease'],(data['diastolic']))
plt.title('Diastolic')

plt.subplot(3,2,5)
sns.boxenplot(data['cardiovascular disease'],(data['age']))
plt.title('Age')

plt.subplot(3,2,6)
sns.boxenplot(data['cardiovascular disease'],(data['bmi']))
plt.title('BMI')

plt.subplots_adjust(hspace= 0.6)
plt.show()

#### 2.8. Heatmap correlation

In [ ]:
corr = data.corr()
f, ax = plt.subplots(figsize = (15,15))
sns.heatmap(corr, annot=True, fmt=".3f", linewidths=0.5, ax=ax)

## 3. Data Analys, Modeling

### 3.1. Tests and Regression

##### 3.1.1. Function for Chi2 tests

In [ ]:
## Fonction Tests du Chi2
def chi2_test(varA, varB):
    h = pd.crosstab(varA, varB, margins=True)
    h
    sns.heatmap(pd.crosstab(varA, varB, normalize=True), 
                annot=True,cmap=sns.cubehelix_palette())
    
    chi2, p, dof, ex = chi2_contingency(h)
    print("chi2 = ", chi2)
    print("p-val = ", p)
    print("degree of freedom = ",dof)

###### Q1: Does gender influence CVDs?

In [ ]:
chi2_test(data['gender'], data['cardiovascular disease'])

###### Q2: Does age have an influence on CVDs?

In [ ]:
chi2_test(data['age'], data['cardiovascular disease'])

###### Q3: Does smoking influence CVDs?

In [ ]:
chi2_test(data['smoke'], data['cardiovascular disease'])

###### Q4: Does physical activity influence CVDs?

In [ ]:
chi2_test(data['active'], data['cardiovascular disease'])

###### Q5: Does alcohol influence CVDs?

In [ ]:
chi2_test(data['alcohol'], data['cardiovascular disease'])

###### Q6: Does cholesterol have an influence on CVDs?

In [ ]:
chi2_test(data['cholesterol'], data['cardiovascular disease'])

###### Q7: Does glucose have an influence on CVDs?

In [ ]:
chi2_test(data['glucose'], data['cardiovascular disease'])

###### Q8: Does bmi have an influence on CVDs?

In [ ]:
chi2_test(data['bmi_class'], data['cardiovascular disease'])

###### Q9: Does bpc have an influence on CVDs?

In [ ]:
chi2_test(data['bpc'], data['cardiovascular disease'])

###### Q10: Does ap_hi have an influence on CVDs?

In [ ]:
chi2_test(data['systolic'], data['cardiovascular disease'])

###### Q11: Does ap_lo have any influence on CVDs?

In [ ]:
chi2_test(data['diastolic'], data['cardiovascular disease'])

##### 3.1.1.2. GLM Estimation

In [ ]:
model = smf.glm(formula = 'cardiovascular disease ~ age+bmi+bpc+smoke+active+alcohol+cholesterol+glucose', 
                data=data, family=sm.families.Binomial())
result = model.fit()
result.summary()

### 3.2. Machine Learning

In [ ]:
data_train = data.drop(['cardiovascular disease'], axis=1)
data_test = data['cardiovascular disease']

train, test, target_train, target_test = train_test_split(data_train, data_test, test_size=0.2, random_state=42)

#### 3.2.1 Machine learning with function

In [ ]:
def predict(var, method, X_train, Y_train, X_test, Y_test):
    var.fit(X_train, Y_train)
    scores = cross_val_score(var, X_train, Y_train, cv=10)
    print('The ',method,' gives an average accuracy of {0:.2f} % with minimun of {1:.2f} % and maximum of {2:.2f} % accuracy'.format(scores.mean() * 100, scores.min() * 100, scores.max() * 100))
    
    Y_pred = var.predict(X_test)
    print(classification_report(Y_test, Y_pred))
    
    plt.rcParams['figure.figsize'] = (5, 5) 
    sns.heatmap(confusion_matrix(Y_test, Y_pred), annot = True, linewidths=.5, cmap="YlGnBu")
    plt.title('Corelation Between Features')
    plt.show()
    
    print('True Positive Cases : {}'.format(confusion_matrix(Y_test, Y_pred)[1][1]))
    print('True Negative Cases : {}'.format(confusion_matrix(Y_test, Y_pred)[0][0]))
    print('False Positive Cases : {}'.format(confusion_matrix(Y_test, Y_pred)[0][1]))
    print('False Negative Cases : {}'.format(confusion_matrix(Y_test, Y_pred)[1][0]))
    
    # Plot ROC curve
    plot_roc_curve(var, X_test, Y_test)
    plt.xlabel('False Positive Rate or (1 - Specifity)')
    plt.ylabel('True Positive Rate or (Sensitivity)')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend(loc="lower right")
    plt.show()

##### 3.2.1.1. Random Forest

In [ ]:
## Random Forest
rfc = RandomForestClassifier(random_state=42, n_estimators=100, max_depth= 10, criterion = 'entropy')
predict(rfc, RandomForestClassifier, train, target_train, test, target_test)

feat_importance = rfc.feature_importances_
feat_importance
plt.rc('xtick', labelsize = 12)
pd.DataFrame({'Feature Importance':feat_importance},
             index=train.columns).sort_values(by='Feature Importance',ascending=True).plot(kind='barh', figsize = (8,8))


##### 3.2.1.2. KNN

In [ ]:
## KNN
knn = KNeighborsClassifier(n_neighbors=50, p=1, weights='uniform')
predict(knn, KNeighborsClassifier, train, target_train, test, target_test)

##### 3.2.1.3. Logistic Regression

In [ ]:
## Regression Logistique 
logreg = LogisticRegression()
predict(logreg, LogisticRegression, train, target_train, test, target_test)

##### 3.2.1.4. Linear SVC

In [ ]:
## Linear SVC
linear_svc = LinearSVC(dual=False)  # dual=False when n_samples > n_features.
predict(linear_svc, LinearSVC, train, target_train, test, target_test)

##### 3.2.1.5. Decision Tree Classifier

In [ ]:
## Decision Tree Classifier
decision_tree = DecisionTreeClassifier()
predict(decision_tree, DecisionTreeClassifier, train, target_train, test, target_test)

##### 3.2.1.6. Gaussian Naive Bayes

In [ ]:
## Gaussian Naive Bayes
gaussian = GaussianNB()
predict(gaussian, GaussianNB, train, target_train, test, target_test)

##### 3.2.1.7. Stochastic Gradient Descent

In [ ]:
## Stochastic Gradient Descent
sgd = SGDClassifier()
predict(sgd, SGDClassifier, train, target_train, test, target_test)